# A-Government Taxation Analysis

**Author:** Victor Prefa  
**Student ID:** 225187913  
**Email:** vprefa@gmail.com  

---

## 📋 Overview

This notebook demonstrates a comprehensive analysis of the A-taxation system, including:

1. **Income Tax Calculation** - Using ATO progressive tax brackets
2. **Weekly Withholding Tax** - Pay-as-you-go (PAYG) calculations
3. **Superannuation Contributions** - Mandatory 11% retirement savings
4. **Tax Refund Estimation** - Comparing withholding vs actual tax liability

The analysis processes 10 simulated employees with varying weekly salaries to demonstrate real-world tax scenarios.

## 📊 Tax Brackets (2024-25 Financial Year)

| Taxable Income | Tax Rate | Calculation |
|----------------|----------|-------------|
| $0 - $18,200 | 0% | Nil (Tax-free threshold) |
| $18,201 - $45,000 | 19% | 19c for each $1 over $18,200 |
| $45,001 - $120,000 | 32.5% | $5,092 + 32.5c for each $1 over $45,000 |
| $120,001 - $180,000 | 37% | $29,467 + 37c for each $1 over $120,000 |
| $180,001+ | 45% | $51,667 + 45c for each $1 over $180,000 |

In [ ]:
# Import required libraries
import numpy as np
import math
from typing import List, Dict, Tuple

## 1️⃣ Generate Sample Weekly Salaries

We generate 10 weekly salaries using a normal distribution with:
- **Mean:** $1,431
- **Standard Deviation:** $527

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate weekly salaries
weekly_salaries_decimal = np.random.normal(1431, 527, 10)
weekly_salaries = [int(round(s)) for s in weekly_salaries_decimal]

print("Generated Weekly Salaries:")
print(f"Decimal: {weekly_salaries_decimal}")
print(f"Rounded: {weekly_salaries}")
print(f"\nRange: ${min(weekly_salaries):,} - ${max(weekly_salaries):,}")
print(f"Mean: ${np.mean(weekly_salaries):,.2f}")

## 2️⃣ Calculate Annual Income Tax

Using the progressive tax bracket system to calculate annual tax liability.

In [ ]:
def calculate_annual_tax(annual_income: float) -> float:
    """
    Calculate annual income tax based on ATO progressive tax brackets.
    
    Args:
        annual_income: Gross annual income in dollars
        
    Returns:
        Annual tax liability in dollars
    """
    if annual_income <= 18200:
        return 0
    elif annual_income <= 45000:
        return 0.19 * (annual_income - 18200)
    elif annual_income <= 120000:
        return 5092 + 0.325 * (annual_income - 45000)
    elif annual_income <= 180000:
        return 29467 + 0.37 * (annual_income - 120000)
    else:
        return 51667 + 0.45 * (annual_income - 180000)

# Convert weekly to annual and calculate tax
annual_incomes = [salary * 52 for salary in weekly_salaries]
income_taxes = [math.ceil(calculate_annual_tax(income)) for income in annual_incomes]

print("Annual Income & Tax Calculations:")
print("=" * 60)
print(f"{'Person':<10} {'Annual Income':>15} {'Income Tax':>15} {'Effective Rate':>15}")
print("-" * 60)
for i, (income, tax) in enumerate(zip(annual_incomes, income_taxes), 1):
    rate = (tax / income * 100) if income > 0 else 0
    print(f"Person {i:<3} ${income:>13,} ${tax:>13,} {rate:>14.1f}%")
print("=" * 60)

## 3️⃣ Calculate Weekly Withholding Tax

Using the ATO withholding formula: **y = a × (x + 0.99) - b**

Where:
- `x` = weekly earnings
- `a` = rate coefficient
- `b` = threshold adjustment

In [ ]:
# Withholding tax coefficient ranges
WITHHOLDING_RANGES = [
    (359, 438, 0.1900, 68.3462),
    (438, 548, 0.2900, 112.1942),
    (548, 721, 0.2100, 68.3465),
    (721, 865, 0.2190, 74.8369),
    (865, 1282, 0.3477, 186.2119),
    (1282, 2307, 0.3450, 182.7504),
    (2307, 3461, 0.3900, 286.5965),
    (3461, float('inf'), 0.4700, 563.5196)
]

def calculate_weekly_withholding(weekly_salary: float) -> float:
    """
    Calculate weekly withholding tax using ATO coefficients.
    
    Formula: y = a × (x + 0.99) - b
    """
    if weekly_salary < 359:
        return 0
    
    for lower, upper, a, b in WITHHOLDING_RANGES:
        if lower <= weekly_salary < upper:
            return round(a * (weekly_salary + 0.99) - b, 2)
    return 0

# Calculate withholding tax for each salary
withholding_taxes = [calculate_weekly_withholding(salary) for salary in weekly_salaries]

print("Weekly Withholding Tax:")
print("=" * 50)
for i, (salary, tax) in enumerate(zip(weekly_salaries, withholding_taxes), 1):
    rate = (tax / salary * 100) if salary > 0 else 0
    print(f"Person {i}: ${salary:,} weekly → ${tax:,.2f} withheld ({rate:.1f}%)")
print("=" * 50)

## 4️⃣ Calculate Tax Refunds

The withholding system is designed to be conservative, often resulting in refunds at year-end.

**Tax Refund = Annual Withholding - Actual Tax Liability**

In [ ]:
# Calculate annual withholding and refunds
annual_withholding = [tax * 52 for tax in withholding_taxes]
tax_refunds = [round(withholding - liability, 2) 
               for withholding, liability in zip(annual_withholding, income_taxes)]

print("Tax Refund Analysis:")
print("=" * 70)
print(f"{'Person':<10} {'Annual Withholding':>18} {'Actual Tax':>15} {'Refund':>15}")
print("-" * 70)
for i, (withholding, tax, refund) in enumerate(zip(annual_withholding, income_taxes, tax_refunds), 1):
    print(f"Person {i:<3} ${withholding:>16,.2f} ${tax:>13,} ${refund:>13,.2f}")
print("-" * 70)
print(f"{'TOTAL':<10} ${sum(annual_withholding):>16,.2f} ${sum(income_taxes):>13,} ${sum(tax_refunds):>13,.2f}")
print("=" * 70)

## 5️⃣ Superannuation Calculations

Superannuation is a mandatory retirement savings scheme. Employers must contribute **11%** of an employee's base salary to their superannuation fund.

When super is included in the salary package:
- Base Salary = Total Package / 1.11
- Super Contribution = Base Salary × 0.11

In [ ]:
SUPER_RATE = 0.11

def calculate_base_and_super(total_salary: float) -> Tuple[float, float]:
    """
    Separate base salary and superannuation from total package.
    """
    base_salary = total_salary / (1 + SUPER_RATE)
    super_amount = base_salary * SUPER_RATE
    return round(base_salary, 2), round(super_amount, 2)

print("Superannuation Breakdown:")
print("=" * 60)
print(f"{'Person':<10} {'Total Package':>15} {'Base Salary':>15} {'Super (11%)':>15}")
print("-" * 60)
for i, salary in enumerate(weekly_salaries, 1):
    base, super_amt = calculate_base_and_super(salary)
    print(f"Person {i:<3} ${salary:>13,} ${base:>13,.2f} ${super_amt:>13,.2f}")
print("=" * 60)

## 6️⃣ Complete Tax Summary Report

Comprehensive breakdown including superannuation-adjusted calculations.

In [ ]:
print("\n" + "=" * 100)
print("                         COMPREHENSIVE TAX ANALYSIS REPORT")
print("=" * 100)

for i, total_weekly in enumerate(weekly_salaries, 1):
    # Calculate base salary and super
    base_weekly, super_weekly = calculate_base_and_super(total_weekly)
    
    # Calculate weekly withholding tax on base salary
    withholding = calculate_weekly_withholding(base_weekly)
    
    # Calculate weekly income (base minus withholding)
    weekly_income = base_weekly - withholding
    
    # Calculate annual amounts
    annual_base = base_weekly * 52
    annual_income_tax = calculate_annual_tax(annual_base)
    annual_withholding = withholding * 52
    tax_return = annual_withholding - annual_income_tax
    
    print(f"\n## Person {i}")
    print(f"   Weekly salary ${total_weekly:.2f}")
    print(f"   Weekly superannuation contribution ${super_weekly:.2f}")
    print(f"   Weekly withholding tax ${withholding:.2f}")
    print(f"   Weekly income ${weekly_income:.2f}")
    print(f"   Income tax ${annual_income_tax:.2f}")
    print(f"   Tax return ${tax_return:.2f}")

print("\n" + "=" * 100)

## 📈 Key Findings & Implications

### For Employees:
1. **Tax-free Threshold**: Income up to $18,200 is not taxed
2. **Progressive System**: Higher earners pay proportionally more tax
3. **Withholding Refunds**: The conservative withholding system typically results in year-end refunds
4. **Superannuation**: Pre-tax super contributions reduce taxable income

### For Employers:
1. **Compliance**: Must withhold correct amounts based on ATO schedules
2. **Super Guarantee**: Mandatory 11% contribution to employee super funds
3. **Competitive Advantage**: Attractive super arrangements can help recruit talent

### Financial Planning:
1. Salary sacrifice to super reduces taxable income
2. Understanding effective tax rates helps with financial planning
3. Tax refunds can be minimized by adjusting withholding if desired

## 📚 References

1. Australian Taxation Office (ATO). (2024). *Tax rates for Australian residents*. https://www.ato.gov.au
2. ATO. (2024). *Tax withheld calculator*. https://www.ato.gov.au/calculators-and-tools/tax-withheld-calculator
3. ATO. (2024). *Super guarantee*. https://www.ato.gov.au/business/super-for-employers